In [1]:
import pandas as pd
import os
current_dir = os.getcwd()
current_dir
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")
current_dir = os.getcwd()
current_dir
#set current working directory to H:\VScode\March Group\March_Team_Project\
os.chdir("H:\\VScode\\March Group\\March_Team_Project\\")

You set a new current directory


In [2]:
# Issue 16 and Issue 15
# Merge weather data with pollution data, ensure pollution data is filtered for locations and timeperiod.
# Merge on date and location
# Save the merged data to a new csv file
# load pollution data
pollution_data = pd.read_csv('Source_Data\\filtered_pollution.zip')
# count of records in pollution data
print(pollution_data.shape)
# filter pollution data for locations and timeperiod
#pollution_filter = pd.read_csv('Outputs/pollution_data_available.csv')
# set pollution filter to list of states in pollution data
pollution_filter = pollution_data[['State']].drop_duplicates()
print(pollution_filter.shape)
# drop rows with missing values
pollution_data = pollution_data.dropna()
# count of records in pollution filter
print(pollution_data.shape)
# filter pollution_data for location in polution_filter with date between years 2003 and 2016
pollution_data = pollution_data[pollution_data['Date Local'].str.contains('200[0-9]|201[0-6]')]
print(pollution_data.shape)
pollution_data = pollution_data[pollution_data['State'].isin(pollution_filter['State'])]
print(pollution_data.shape)
pollution_data
# drop Unnamed: 0 column
pollution_data = pollution_data.drop(columns=['Unnamed: 0'])
# summarize pollution data
pollution_data_summary = pollution_data.groupby(['State Code',
                                                 'County Code',
                                                 'Site Num',
                                                 'Address',
                                                 'State',
                                                 'County',
                                                 'City',
                                                 'Date Local',
                                                 'Year']).agg(
                                                     {'NO2 Mean': 'mean',
                                                      'NO2 1st Max Value': 'max',
                                                      'NO2 1st Max Hour': 'max',
                                                      'NO2 AQI': 'mean',
                                                      'O3 Mean': 'mean',
                                                      'O3 1st Max Value': 'max',
                                                      'O3 1st Max Hour': 'max',
                                                      'O3 AQI': 'mean',
                                                      'SO2 Mean': 'mean',
                                                      'SO2 1st Max Value': 'max',
                                                      'SO2 1st Max Hour': 'max',
                                                      'SO2 AQI': 'mean',
                                                      'CO Mean': 'mean',
                                                      'CO 1st Max Value': 'max',
                                                      'CO 1st Max Hour': 'max',
                                                      'CO AQI': 'mean'
                                                     }).reset_index()
# summarize pollution data
print(pollution_data_summary.shape)


(878851, 30)
(13, 1)
(219827, 30)
(219827, 30)
(219827, 30)
(213308, 25)


In [4]:
# provide a count of records by year
pollution_data['year'] = pollution_data['Date Local'].str[:4]
pollution_data.groupby('year').size()   


year
2003    18995
2004    18406
2005    19553
2006    20097
2007    22284
2008    23203
2009    19668
2010    18131
2011    19824
2012    19029
2013    20637
dtype: int64

In [3]:
# Create a cleaned version of pollution data where any row with a null value is removed
pollution_data_cleaned = pollution_data.dropna()
print(f"Shape after dropping null values: {pollution_data_cleaned.shape}")

# Format 'City' column as uppercase for consistency
pollution_data_cleaned['City'] = pollution_data_cleaned['City'].str.upper()

# Rename 'Date Local' to 'Date' for better readability and consistency
pollution_data_cleaned.rename(columns={'Date Local': 'Date'}, inplace=True)

# Convert 'Date' column to datetime format for easier date manipulation
pollution_data_cleaned['Date'] = pd.to_datetime(pollution_data_cleaned['Date'])

# Display the first few rows of the cleaned data for verification
pollution_data_cleaned.head()
print(pollution_data_cleaned.shape)

Shape after dropping null values: (219827, 29)
(219827, 29)


In [ ]:
# load weather data on day and city
weather_data = pd.read_csv('Not_to_be_shared_to_repo//Us_Weather_Final_10km_V2.zip')
# rename columns to match stations
weather_data = weather_data.rename(columns={'Station_ID':'StationId'})
# convert date to datetime current format yyyymmdd
weather_data['Date'] = weather_data['Date'].astype(str)
weather_data['Date'] = weather_data['Date'].apply(lambda x: x[:4] + '-' + x[4:6] + '-' + x[6:])
weather_data['Date'] = pd.to_datetime(weather_data['Date'])
# drop columns not needed
weather_data = weather_data.drop(columns=["Unnamed: 0", "Unnamed: 0.1"])
print(weather_data.shape)
weather_data

MemoryError: Unable to allocate 6.44 GiB for an array with shape (36, 24010324) and data type float64

In [9]:
# load Us_Stations_with_City_10km.csv
stations = pd.read_csv('Source_Data/Us_Stations_with_City_100km.csv')
# get siteId from StationId - last 4 characters
stations['SiteId'] = stations['StationId'].str[-8:]
# format StationId as capitals
stations['StationId'] = stations['StationId'].str.upper()
# format city as capitals
stations['City'] = stations['City'].str.upper()
print(stations.shape)
stations


(21979, 9)


,StationId,Latitude,Longitude,Elevation,Name,CountryCode,City,CityDistance,SiteId
0,US10BOYD002,42.9095,-98.8492,552.9,BUTTE 0.1 S,US,DALLAS,65.421913,0boyd002
1,US10BOYD003,42.9141,-98.6603,513.0,SPENCER 3.5 NE,US,DALLAS,78.381858,0boyd003
2,US10BOYD005,42.9096,-98.8428,555.0,BUTTE 0.4 ESE,US,DALLAS,65.847671,0boyd005
3,US10BROW001,42.5359,-99.7092,791.9,LONG PINE 0.4 W,US,DALLAS,79.605910,0brow001
4,US10BROW005,42.6748,-99.7695,686.1,LONG PINE 15.7 ESE,US,DALLAS,65.886702,0brow005
...,...,...,...,...,...,...,...,...,...
21974,USW00094978,41.7631,-96.1797,312.1,TEKAMAH MUNI AP,US,WASHINGTON,40.707000,00094978
21975,USW00094982,41.6117,-90.5808,259.4,DAVENPORT,US,WASHINGTON,99.014447,00094982
21976,USW00094990,43.3892,-99.8433,615.1,WINNER WILEY FLD,US,DALLAS,31.270658,00094990
21977,USW00094995,40.8483,-96.5650,362.4,LINCOLN 8 ENE,US,WASHINGTON,68.028597,00094995


In [10]:
# merge weather data with stations on station id
weather_data = pd.merge(weather_data, stations, on='StationId')
print(weather_data.shape)
# set all null values to 0
weather_data = weather_data.fillna(0)
# rename columns Latitude and Longitude to WS_Latitude and WS_Longitude
weather_data = weather_data.rename(columns={'Latitude':'WS_Latitude', 'Longitude':'WS_Longitude', "Elevation": "WS_Elevation", "Name": "WS_Name"})
# filter to only keep 1 record per city per day based on the closest station
print(weather_data.shape)

weather_data

(380584, 31)
(380584, 31)


,StationId,Date,AWND,PGTM,PRCP,TAVG,TMAX,TMIN,TSUN,WDMV,...,WT18,WT22,WS_Latitude,WS_Longitude,WS_Elevation,WS_Name,CountryCode,City,CityDistance,SiteId
0,USC00010764,2003-01-01,0.0,0.0,36.0,0.0,144.0,39.0,0.0,0.0,...,0.0,0.0,33.3936,-87.0067,132.6,BESSEMER 3 WSW,US,CONCORD,8.742848,00010764
1,USC00010764,2003-01-02,0.0,0.0,30.0,0.0,106.0,28.0,0.0,0.0,...,0.0,0.0,33.3936,-87.0067,132.6,BESSEMER 3 WSW,US,CONCORD,8.742848,00010764
2,USC00010764,2003-01-03,0.0,0.0,0.0,0.0,39.0,-11.0,0.0,0.0,...,0.0,0.0,33.3936,-87.0067,132.6,BESSEMER 3 WSW,US,CONCORD,8.742848,00010764
3,USC00010764,2003-01-04,0.0,0.0,0.0,0.0,111.0,-44.0,0.0,0.0,...,0.0,0.0,33.3936,-87.0067,132.6,BESSEMER 3 WSW,US,CONCORD,8.742848,00010764
4,USC00010764,2003-01-05,0.0,0.0,0.0,0.0,167.0,22.0,0.0,0.0,...,0.0,0.0,33.3936,-87.0067,132.6,BESSEMER 3 WSW,US,CONCORD,8.742848,00010764
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380579,US1VAAP0001,2013-12-23,0.0,0.0,244.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,37.4083,-78.9638,219.5,CONCORD 4.6 NNE,US,CONCORD,7.568900,VAAP0001
380580,US1VAAP0001,2013-12-24,0.0,0.0,64.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,37.4083,-78.9638,219.5,CONCORD 4.6 NNE,US,CONCORD,7.568900,VAAP0001
380581,US1VAAP0001,2013-12-29,0.0,0.0,163.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,37.4083,-78.9638,219.5,CONCORD 4.6 NNE,US,CONCORD,7.568900,VAAP0001
380582,US1VAAP0001,2013-12-30,0.0,0.0,157.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,37.4083,-78.9638,219.5,CONCORD 4.6 NNE,US,CONCORD,7.568900,VAAP0001


In [ ]:
# filter pollution data to have 1 record per city per day using the record with the lowest CityDistance
print(pollution_data_cleaned.shape)
# merge pollution data with weather data on date and location only take first match
merged_data = pd.merge(pollution_data_cleaned, weather_data, on=['Date', 'City'], how='inner')
merged_data = merged_data.sort_values(by=['Date', 'City', 'CityDistance']).drop_duplicates(subset=['Date', 'City'], keep='first')
print(merged_data.shape)
merged_data

(219827, 30)
(46054, 59)


,State Code,County Code,Site Num,Address,State,County,City,Date,NO2 Units,NO2 Mean,...,WT16,WT18,WT22,WS_Latitude,WS_Longitude,WS_Elevation,WS_Name,CountryCode,CityDistance,SiteId
9811,22,33,9,1061-A Leesville Ave,Louisiana,East Baton Rouge,BATON ROUGE,2003-01-01,Parts per billion,14.652174,...,0.0,0.0,0.0,30.4192,-91.1303,15.2,BATON ROUGE CONCORD,US,2.536404,00160548
11243,25,25,42,HARRISON AVE,Massachusetts,Suffolk,BOSTON,2003-01-01,Parts per billion,23.363636,...,0.0,0.0,0.0,37.5844,-75.8217,9.1,PAINTER 2 W,US,2.908371,00446475
13582,34,7,3,COPEWOOD E. DAVIS STS; TRAILER,New Jersey,Camden,CAMDEN,2003-01-01,Parts per billion,32.750000,...,0.0,0.0,0.0,36.0519,-88.0831,128.9,CAMDEN,US,2.516716,00401352
598,6,13,2,2956-A TREAT BOULEVARD,California,Contra Costa,CONCORD,2003-01-01,Parts per billion,12.913043,...,1.0,0.0,1.0,43.2050,-71.5025,103.0,CONCORD ASOS,US,5.419824,00014745
16429,48,113,69,1415 Hinton Street,Texas,Dallas,DALLAS,2003-01-01,Parts per billion,18.583333,...,0.0,0.0,0.0,35.2672,-81.1436,213.4,GASTONIA,US,6.582411,00313356
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385695,36,81,124,Queens College 65-30 Kissena Blvd Parking L...,New York,Queens,NEW YORK,2013-12-31,Parts per billion,19.287500,...,0.0,0.0,0.0,40.7145,-73.8819,32.3,MIDDLE VILLAGE 0.5 SW,US,4.264213,NYQN0002
389906,42,101,4,1501 E. LYCOMING AVE.,Pennsylvania,Philadelphia,PHILADELPHIA,2013-12-31,Parts per billion,19.505263,...,0.0,0.0,0.0,32.7692,-89.1303,125.9,PHILA 1 WSW,US,1.084637,00226894
332532,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,PHOENIX,2013-12-31,Parts per billion,32.695652,...,0.0,0.0,0.0,43.1951,-76.2883,118.9,BALDWINSVILLE 3.5 NE,US,4.107544,NYOG0046
365158,11,1,41,"420 34th Street N.E.,Washington, DC 20019",District Of Columbia,District of Columbia,WASHINGTON,2013-12-31,Parts per billion,8.272727,...,0.0,0.0,0.0,39.8089,-97.0550,397.8,WASHINGTON,US,0.955429,00148578


In [13]:
# count of records By Date
datecount = merged_data.groupby('Date').size()
datecount

Date
2003-01-01    11
2003-01-02    11
2003-01-03    11
2003-01-04    11
2003-01-05    11
              ..
2013-12-27    14
2013-12-28    14
2013-12-29    14
2013-12-30    14
2013-12-31    14
Length: 4018, dtype: int64

In [14]:
# count unique pollution sites per day
pollution_sites = merged_data.groupby('Date')['Site Num'].nunique()
pollution_sites

Date
2003-01-01    11
2003-01-02    11
2003-01-03    11
2003-01-04    11
2003-01-05    11
              ..
2013-12-27    13
2013-12-28    13
2013-12-29    13
2013-12-30    13
2013-12-31    13
Name: Site Num, Length: 4018, dtype: int64

In [15]:
# deduplicate merged data to provide 1 row per Date, city, county, state, address, site num, county code, state code, choosing record with lowest CityDistance if multiple records exist
merged_data_sort = merged_data.sort_values(by=['Date','State', 'Address', 'County', 'Site Num', 'City', 'CityDistance']).copy()
print(merged_data_sort.shape)
merged_data_dedupe = merged_data_sort.drop_duplicates(subset=['Date', 'City', 'County', 'State', 'Address', 'Site Num', 'County Code', 'State Code'], keep='first')
print(merged_data_dedupe.shape)


(46054, 59)
(46054, 59)


In [16]:
#  display Percentage of null values in each column
null_values_percentage = merged_data_dedupe.isnull().sum() / len(merged_data) * 100
null_values_percentage


State Code           0.0
County Code          0.0
Site Num             0.0
Address              0.0
State                0.0
County               0.0
City                 0.0
Date                 0.0
NO2 Units            0.0
NO2 Mean             0.0
NO2 1st Max Value    0.0
NO2 1st Max Hour     0.0
NO2 AQI              0.0
O3 Units             0.0
O3 Mean              0.0
O3 1st Max Value     0.0
O3 1st Max Hour      0.0
O3 AQI               0.0
SO2 Units            0.0
SO2 Mean             0.0
SO2 1st Max Value    0.0
SO2 1st Max Hour     0.0
SO2 AQI              0.0
CO Units             0.0
CO Mean              0.0
CO 1st Max Value     0.0
CO 1st Max Hour      0.0
CO AQI               0.0
Year                 0.0
year                 0.0
StationId            0.0
AWND                 0.0
PGTM                 0.0
PRCP                 0.0
TAVG                 0.0
TMAX                 0.0
TMIN                 0.0
TSUN                 0.0
WDMV                 0.0
WT01                 0.0


In [17]:
# Apply AQI classifications to the data
# AQI = 0-50 Good, 51-100 Moderate, 101-150 Unhealthy for Sensitive Groups, 151-200 Unhealthy, 201-300 Very Unhealthy, 301-500 Hazardous
# Create a new column AQI_Range
# Create a new column AQI_Range_Description

def AQI_Range(value):
    if value >= 0 and value <= 50:
        return 'Good'
    if value >= 51 and value <= 100:
        return 'Moderate'
    if value >= 101 and value <= 150:
        return 'Unhealthy for Sensitive Groups'
    if value >= 151 and value <= 200:
        return 'Unhealthy'
    if value >= 201 and value <= 300:
        return 'Very Unhealthy'
    if value >= 301 and value <= 500:
        return 'Hazardous'
    return 'Unknown'

def AQI_Range_Description(value):
    if value >= 0 and value <= 50:
        return 'Air quality is considered satisfactory, and air pollution poses little or no risk.'
    if value >= 51 and value <= 100:
        return 'Air quality is acceptable; however, for some pollutants there may be a moderate health concern for a very small number of people who are unusually sensitive to air pollution.'
    if value >= 101 and value <= 150:
        return 'Members of sensitive groups may experience health effects. The general public is not likely to be affected.'
    if value >= 151 and value <= 200:
        return 'Everyone may begin to experience health effects; members of sensitive groups may experience more serious health effects.'
    if value >= 201 and value <= 300:
        return 'Health alert: everyone may experience more serious health effects.'
    if value >= 301 and value <= 500:
        return 'Health warnings of emergency conditions. The entire population is more likely to be affected.'
    return 'Unknown'

def AQI_Group(value):
    if value >= 0 and value <= 50:
        return 1
    if value >= 51 and value <= 100:
        return 2
    if value >= 101 and value <= 150:
        return 3
    if value >= 151 and value <= 200:
        return 4
    if value >= 201 and value <= 300:
        return 5
    if value >= 301 and value <= 500:
        return 6
    return 'Unknown'

# apply AQI_Range, AQI_Group and AQI_Range_Description to O3_AQI, SO2_AQI, CO_AQI, NO2_AQI
merged_data['O3_AQI_Range'] = merged_data['O3 AQI'].apply(AQI_Range)
merged_data['O3_AQI_Range_Description'] = merged_data['O3 AQI'].apply(AQI_Range_Description)
merged_data['O3_AQI_Group'] = merged_data['O3 AQI'].apply(AQI_Group)
merged_data['SO2_AQI_Range'] = merged_data['SO2 AQI'].apply(AQI_Range)
merged_data['SO2_AQI_Range_Description'] = merged_data['SO2 AQI'].apply(AQI_Range_Description)
merged_data['SO2_AQI_Group'] = merged_data['SO2 AQI'].apply(AQI_Group)
merged_data['CO_AQI_Range'] = merged_data['CO AQI'].apply(AQI_Range)
merged_data['CO_AQI_Range_Description'] = merged_data['CO AQI'].apply(AQI_Range_Description)
merged_data['CO_AQI_Group'] = merged_data['CO AQI'].apply(AQI_Group)
merged_data['NO2_AQI_Range'] = merged_data['NO2 AQI'].apply(AQI_Range)
merged_data['NO2_AQI_Range_Description'] = merged_data['NO2 AQI'].apply(AQI_Range_Description)
merged_data['NO2_AQI_Group'] = merged_data['NO2 AQI'].apply(AQI_Group)

merged_data

,State Code,County Code,Site Num,Address,State,County,City,Date,NO2 Units,NO2 Mean,...,O3_AQI_Group,SO2_AQI_Range,SO2_AQI_Range_Description,SO2_AQI_Group,CO_AQI_Range,CO_AQI_Range_Description,CO_AQI_Group,NO2_AQI_Range,NO2_AQI_Range_Description,NO2_AQI_Group
9811,22,33,9,1061-A Leesville Ave,Louisiana,East Baton Rouge,BATON ROUGE,2003-01-01,Parts per billion,14.652174,...,1,Good,"Air quality is considered satisfactory, and ai...",1,Good,"Air quality is considered satisfactory, and ai...",1,Good,"Air quality is considered satisfactory, and ai...",1
11243,25,25,42,HARRISON AVE,Massachusetts,Suffolk,BOSTON,2003-01-01,Parts per billion,23.363636,...,1,Good,"Air quality is considered satisfactory, and ai...",1,Good,"Air quality is considered satisfactory, and ai...",1,Good,"Air quality is considered satisfactory, and ai...",1
13582,34,7,3,COPEWOOD E. DAVIS STS; TRAILER,New Jersey,Camden,CAMDEN,2003-01-01,Parts per billion,32.750000,...,1,Good,"Air quality is considered satisfactory, and ai...",1,Good,"Air quality is considered satisfactory, and ai...",1,Good,"Air quality is considered satisfactory, and ai...",1
598,6,13,2,2956-A TREAT BOULEVARD,California,Contra Costa,CONCORD,2003-01-01,Parts per billion,12.913043,...,1,Good,"Air quality is considered satisfactory, and ai...",1,Good,"Air quality is considered satisfactory, and ai...",1,Good,"Air quality is considered satisfactory, and ai...",1
16429,48,113,69,1415 Hinton Street,Texas,Dallas,DALLAS,2003-01-01,Parts per billion,18.583333,...,1,Good,"Air quality is considered satisfactory, and ai...",1,Good,"Air quality is considered satisfactory, and ai...",1,Moderate,"Air quality is acceptable; however, for some p...",2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385695,36,81,124,Queens College 65-30 Kissena Blvd Parking L...,New York,Queens,NEW YORK,2013-12-31,Parts per billion,19.287500,...,1,Good,"Air quality is considered satisfactory, and ai...",1,Good,"Air quality is considered satisfactory, and ai...",1,Good,"Air quality is considered satisfactory, and ai...",1
389906,42,101,4,1501 E. LYCOMING AVE.,Pennsylvania,Philadelphia,PHILADELPHIA,2013-12-31,Parts per billion,19.505263,...,1,Good,"Air quality is considered satisfactory, and ai...",1,Good,"Air quality is considered satisfactory, and ai...",1,Good,"Air quality is considered satisfactory, and ai...",1
332532,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,PHOENIX,2013-12-31,Parts per billion,32.695652,...,1,Good,"Air quality is considered satisfactory, and ai...",1,Good,"Air quality is considered satisfactory, and ai...",1,Good,"Air quality is considered satisfactory, and ai...",1
365158,11,1,41,"420 34th Street N.E.,Washington, DC 20019",District Of Columbia,District of Columbia,WASHINGTON,2013-12-31,Parts per billion,8.272727,...,1,Good,"Air quality is considered satisfactory, and ai...",1,Good,"Air quality is considered satisfactory, and ai...",1,Good,"Air quality is considered satisfactory, and ai...",1


In [ ]:
# Count records per city - 
# Group by city and count records
city_count = merged_data.groupby(['State', 'City']).size()
expected = 365*10
city_count

State                 City        
Arizona               PHOENIX         3900
California            CONCORD         3984
                      LONG BEACH      3819
                      LOS ANGELES     3678
District Of Columbia  WASHINGTON      3967
Florida               WINTER PARK     3958
Kansas                KANSAS CITY     3724
Louisiana             BATON ROUGE     3954
Massachusetts         BOSTON          3916
New Jersey            CAMDEN          2517
New York              NEW YORK         915
Pennsylvania          PHILADELPHIA    3815
Texas                 DALLAS          3450
                      EL PASO          457
dtype: int64

In [19]:
# Save merged to zip file
merged_data.to_csv('Outputs/DashBoardData.zip', index=False)